# Memory-Efficient Backpropagation Through Time

* 싸이그래머 / IAM : 파트 1 - 딥마인드 논문 리뷰 [1]
* 김무성

# Contents
* 1 Introduction
* 2 Background and related work
    - 2.1 Backpropagation through Time
    - 2.2 Trading memory for computation time
* 3 Memory-efficient backpropagation through time
    - 3.1 Backpropagation though time with selective hidden state memorization (BPTT-HSM)
    - 3.2 Backpropagation though time with selective internal state memorization (BPTT-ISM)
    - 3.3 Backpropagation though time with mixed state memorization (BPTT-MSM)
    - 3.4 Removing double hidden-state memorization
    - 3.5 Analytical upper bound for BPTT-HSM
    - 3.6 Comparison of the three different strategies
* 4 Discussion
    - 4.1 Optimality
    - 4.2 Numerical comparison with Chen's $\sqrt{t}$ algorithm

#### Abstract
* We propose a novel approach to reduce memory consumption of the backpropagation through time (BPTT) algorithm when training recurrent neural networks (RNNs). 
* Our approach uses <font color="red">dynamic programming to balance a trade-off between caching of intermediate results and recomputation</font>. The algorithm is capable of tightly fitting within almost any user-set memory budget while finding an optimal execution policy minimizing the computational cost.
* Computational devices have limited memory capacity and maximizing a computational performance given a fixed memory budget is a practical use-case. 
* We provide asymptotic computational upper bounds for various regimes. 
* The algorithm is particularly effective for long sequences. For sequences of length 1000, our algorithm saves 95% of memory usage while using only one third more time per iteration than the standard BPTT.

# 1 Introduction

* Backpropagation Through Time algorithm (BPTT) (Rumelhart et al. [1985], Werbos [1990]) is typically used to obtain gradients during training. One important problem is the large memory consumption required by the BPTT. This is especially troublesome when using Graphics Processing Units (GPUs) due to the limitations of GPU memory.
* Memory budget is typically known in advance. Our algorithm balances the tradeoff between memo- rization and recomputation by finding an optimal memory usage policy which minimizes the total computational cost for any fixed memory budget. The algorithm exploits the fact that the same memory slots may be reused multiple times. The idea to use dynamic programming to find a provably optimal policy is the main contribution of this paper.

# 2 Background and related work
* 2.1 Backpropagation through Time
* 2.2 Trading memory for computation time

<img src="http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg" width=600 />

#### Definition 1. An RNN core 
* is a feed-forward neural network which is cloned (unfolded in time) repeatedly, where each clone represents a particular time point in the recurrence.
* For example, if an RNN has a single hidden layer whose outputs feed back into the same hidden layer, then for a sequence length of t the unfolded network is feed-forward and contains t RNN cores.

#### Definition 2. The hidden state 
* of the recurrent network is the part of the output of the RNN core which is passed into the next RNN core as an input.
* In addition to the initial hidden state, there exists a single hidden state per time step once the network is unfolded.

#### Definition 3. The internal state 
* of the RNN core for a given time-point is all the necessary information required to backpropagate gradients over that time step once an input vector, a gradient with respect to the output vector, and a gradient with respect to the output hidden state is supplied. We define it to also include an output hidden state.


## 2.1 Backpropagation through Time

<img src="https://i.ytimg.com/vi/Nj2ab1PzoEI/maxresdefault.jpg" width=600 />

<img src="https://pbs.twimg.com/media/CQ0CJtwUkAAL__H.png" width=600 />

## 2.2 Trading memory for computation time

* The general idea of <font color="red">trading computation time and memory consumption in general computation graphs</font> has been investigated in the automatic differentiation community (Dauvergne and Hascoët [2006]). 
* Recently, the rise of deep architectures and recurrent networks has increased interest in a less general case where the graph of forward computation is a chain and gradients have to be chained in a reverse order. This simplification leads to relatively simple memory-saving strategies and heuristics. 
* In the context of BPTT, <font color="red">instead of storing hidden network states</font>, some of the <font color="red">intermediate results can be recomputed</font> <font color="blue">on demand by executing an extra forward operation</font>.

#### Chen’s $\sqrt{t}$ algorithm
* O($\sqrt{t}$)


#### Chen’s recursive algorithm.
* O($klog_{k+1}(t)$)
    - O($tlog_k(t)$).
* $log_2(t)$ in the case when k = 1.

# 3 Memory-efficient backpropagation through time
* 3.1 Backpropagation though time with selective hidden state memorization (BPTT-HSM)
* 3.2 Backpropagation though time with selective internal state memorization (BPTT-ISM)
* 3.3 Backpropagation though time with mixed state memorization (BPTT-MSM)
* 3.4 Removing double hidden-state memorization
* 3.5 Analytical upper bound for BPTT-HSM
* 3.6 Comparison of the three different strategies

We first discuss two simple examples: 
* when memory is very scarce, 
    - This would require repeating t forward steps before backpropagating gradients one step backwards (<font color="red">we only remember inputs and the initial state</font>).
    - This would produce an algorithm requiring t(t + 1)/2 forward passes to backpropagate errors over t time steps. 
    - The algorithm would be O(1) in space and $O(t^2)$ in time.
* when it is somewhat limited
    - A simple way of reducing memory consumption is <font color="red">simply to store only hidden RNN states at all time points</font>. 
    - When errors have to be backpropagated from time t to t − 1, an internal RNN core state can be re-evaluated by executing another forward operation taking the previous hidden state as an input. The backward operation can follow immediately. 
    - This approach can lead to fairly significant memory savings, as typically the recurrent network hidden state is much smaller than an internal state of the network core itself. 
    - <font color="red">On the other hand this leads to another forward operation being executed during the backpropagation stage</font>.

## 3.1 Backpropagation though time with selective hidden state memorization (BPTT-HSM)

The idea behind the proposed algorithm is to compromise between two previous extremes. Suppose
that we want to forward and backpropagate a sequence of length t, but we are only able to store m
hidden states in memory at any given time. We may reuse the same memory slots to store different
hidden states during backpropagation. Also, suppose that we have a single RNN core available for
the purposes of intermediate calculations which is able to store a single internal state.

<img src="figures/cap1.png" width=600 />

<img src="figures/cap2.png" width=600 />

<img src="figures/cap3.png" width=600 />

<img src="figures/cap4.png" width=600 />

## 3.2 Backpropagation though time with selective internal state memorization (BPTT-ISM)

Saving internal RNN core states instead of hidden RNN states would allow us to save a single forward operation during backpropagation in every divide-and-conquer step, but at a higher memory cost.

<img src="figures/cap5.png" width=600 />

<img src="figures/cap6.png" width=600 />

<img src="figures/cap7.png" width=600 />

<img src="figures/cap8.png" width=600 />

## 3.3 Backpropagation though time with mixed state memorization (BPTT-MSM)

<img src="figures/cap9.png" width=600 />

We can solve the above equations by using simple dynamic programming. H(t,m) will indicate whether the next state to be pushed into memory in a hidden state or an internal state, while the respective values if D1 (t, m) and D2 (t, m) will indicate the position of the next push.


## 3.4 Removing double hidden-state memorization

Definition 3 of internal RNN core state would typically require for a hidden input state to be included for each memorization. This may lead to the duplication of information. For example, when an optimal strategy is to remember a few internal RNN core states in sequence, a memorized hidden output of one would be equal to a memorized hidden input for the other one (see Definition 3).

Every time we want to push an internal RNN core state onto the stack and a previous internal state is already there, we may omit pushing the input hidden state. Recall that an internal core RNN state when an input hidden state is otherwise not known is α times larger than a hidden state. Define β ≤ α as the space required to memorize the internal core state when an input hidden state is known. A relationship between α and β is application-specific, but in many circumstances α = β + 1. We only have to modify (7) to reflect this optimization:

<img src="figures/cap10.png" width=600 />

## 3.5 Analytical upper bound for BPTT-HSM

## 3.6 Comparison of the three different strategies

<img src="figures/cap11.png" width=600 />

Computational costs for each previously described strategy and the results are shown in Figure 6. BPTT-MSM outperforms both BPTT-ISM and BPTT-HSM. This is unsurprising, because the search space in that case is a superset of both strategy spaces, while the algorothm finds an optimal strategy within that space. Also, for a fixed memory capacity, the strategy memorizing only hidden states outperforms a strategy memorizing internal RNN core states for long sequences, while the latter outperforms the former for relatively short sequences

# 4 Discussion
* 4.1 Optimality
* 4.2 Numerical comparison with Chen's $\sqrt{t}$ algorithm

## 4.1 Optimality

The dynamic program finds the optimal computational strategy by construction, subject to memory constraints and a fairly general model that we impose. As both strategies proposed by Chen et al. 

## 4.2 Numerical comparison with Chen's $\sqrt{t}$ algorithm

In other words, the model does not allow for a fine-grained control over memory usage and rather saves some memory. In the meantime our proposed BPTT-MSM can fit within almost arbitrary constant memory constraints, and this is the main advantage of our algorithm.

<img src="figures/cap12.png" width=600 />

# 참고자료
* [1] Memory-Efficient Backpropagation Through Time -  https://arxiv.org/abs/1606.03401